In [21]:
import json
import pandas as pd

In [22]:
def convert(from_currency, to_currency, amount): 
    initial_amount = amount 

    #load exchange rates
    with open("currency.json") as currency:
        c = json.load(currency)
        c = c[to_currency]

    if from_currency != 'AED' : 
        amount = round(amount / c[from_currency] , 4)
    return amount

In [31]:
df = pd.read_csv('bookings_data.csv', parse_dates=['SalesDate', 'InDate', 'OutDate', 'CreateDate', 'LastProcessDate', 'CancelDate'])
df = df.drop_duplicates(subset='RefID', keep='last')


In [32]:
for col in df.columns:
    if col in ["GuestName", "RoomDescription"]:
        df[col] = df[col].str.upper()
    elif col in ["P_Price", "S_Price"]:
        df[col] = df[col].fillna(0)
    elif col in ["P_Currency", "S_Currency"]:
        df[col] = df[col].fillna("AED")
    elif col == "Statu":
        df[col] = df[col].str.capitalize()

df.loc[df.P_Price < 1, "P_Price"] = 0
df.loc[df.S_Price < 1, "S_Price"] = 0
df.loc[df.CancelDate == "2000-01-01", "CancelDate"] = None
df.head()

,RefID,ResID,HotelID,OperatorID,VoucherID,GuestName,SalesDate,InDate,OutDate,RoomDescription,...,P_Price,P_Currency,S_Price,S_Currency,PurchaseIndicator,SalesIndicator,CreateDate,LastProcessDate,CancelDate,Statu
0,1768502,98750,125938,81,1371309,BORRELLBOTELLA JORGE,2013-09-12,2021-09-16,2021-09-19,DELUXE TWIN,...,0.0,AED,0.0,AED,Normal,Normal,2021-09-03,2021-09-03,NaT,Ok
1,1349962,1134726,127537,1426,LELAQUET JEAN1,LELAQUET JEAN1,2019-04-24,2021-01-10,2021-01-17,1 BEDROOM APARTMENT PROMO,...,0.0,AED,0.0,AED,Special,Special,2019-04-24,2021-01-08,2021-01-08,Can
2,1378725,1159349,166654,6,854MU,INGO MAERKERT,2019-06-21,2021-04-20,2021-05-01,DELUXE TWIN RIVERVIEW,...,0.0,AED,0.0,AED,Special,Special,2019-06-27,2021-01-30,2021-01-30,Can
3,1380403,1160766,152016,38,DEAN JARVIS1,DEAN JARVIS1,2019-07-01,2021-05-29,2021-06-05,JUNIORSUITE GARDENVIEW,...,0.0,AED,0.0,AED,Special,Special,2019-07-01,2021-01-14,2021-01-14,Can
4,1625790,1172544,191613,69,3988436,ISABEL LAZODENEUBER,2019-07-26,2021-06-07,2021-06-09,DELUXE TWIN SEAVIEW,...,0.0,AED,0.0,AED,Special,Special,2020-08-20,2021-05-08,2021-05-08,Can


In [27]:
df['P_Price'] = df.apply(lambda x: convert(x.P_Currency, "AED", x.P_Price), axis=1 )
df['S_Price'] = df.apply(lambda x: convert(x.S_Currency, "AED", x.S_Price), axis=1 )

In [28]:
df.to_csv('data.csv', index=False)